In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df=pd.read_csv('/kaggle/input/cogo-insurance/FAF5.4_2018-2020.csv')

In [3]:
commodity_labels_df=pd.read_csv('/kaggle/input/cogo-insurance/comodity-labels.csv')
location_labels_df=pd.read_csv('/kaggle/input/cogo-insurance/location-labels.csv')

In [4]:
df.head()

,fr_orig,dms_orig,dms_dest,fr_dest,fr_inmode,dms_mode,fr_outmode,sctg2,trade_type,dist_band,...,tons_2020,value_2018,value_2019,value_2020,current_value_2018,current_value_2019,current_value_2020,tmiles_2018,tmiles_2019,tmiles_2020
0,NaN,11,11,NaN,NaN,1,NaN,1,1,1,...,54.395645,68.594362,71.436584,71.943453,66.560261,67.993017,62.808675,3.218410,3.351765,3.375547
1,NaN,11,19,NaN,NaN,1,NaN,1,1,2,...,411.215079,518.553218,540.039582,543.871367,503.176016,514.007225,474.815125,48.631191,50.646235,51.005589
2,NaN,11,129,NaN,NaN,1,NaN,1,1,3,...,1.450736,1.829418,1.905220,1.918738,1.775168,1.813380,1.675113,0.450066,0.468715,0.472040
3,NaN,11,131,NaN,NaN,1,NaN,1,1,2,...,13.318528,16.795020,17.490925,17.615030,16.296980,16.647783,15.378421,2.177036,2.267241,2.283328
4,NaN,11,139,NaN,NaN,1,NaN,1,1,2,...,5.475181,6.904350,7.190433,7.241452,6.699608,6.843822,6.321993,1.274964,1.327792,1.337213


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2046411 entries, 0 to 2046410
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   fr_orig             float64
 1   dms_orig            int64  
 2   dms_dest            int64  
 3   fr_dest             float64
 4   fr_inmode           float64
 5   dms_mode            int64  
 6   fr_outmode          float64
 7   sctg2               int64  
 8   trade_type          int64  
 9   dist_band           int64  
 10  tons_2018           float64
 11  tons_2019           float64
 12  tons_2020           float64
 13  value_2018          float64
 14  value_2019          float64
 15  value_2020          float64
 16  current_value_2018  float64
 17  current_value_2019  float64
 18  current_value_2020  float64
 19  tmiles_2018         float64
 20  tmiles_2019         float64
 21  tmiles_2020         float64
dtypes: float64(16), int64(6)
memory usage: 343.5 MB


## Preprocessing

In [6]:
## Decoding trade type
trade_type_dict={1:'Domestic only',2:'Import',3:'Export'}
df['trade_type'].replace(trade_type_dict,inplace=True)

In [7]:
## Replacing domestic location id's with their actual location

location_dict={}
for i in range(len(location_labels_df)):
    id=location_labels_df.loc[i]['id']
    region=location_labels_df.loc[i]['region']
    location_dict.update({id:region})
    
df['orig_region']=df['dms_orig'].replace(location_dict)
df['dest_region']=df['dms_dest'].replace(location_dict)

In [8]:
## Creating state column based on region id's with the help of locations csv file

location_dict={}
for i in range(len(location_labels_df)):
    id=location_labels_df.loc[i]['id']
    state=location_labels_df.loc[i]['state']
    location_dict.update({id:state})
    
df['orig_state']=df['dms_orig'].replace(location_dict)
df['dest_state']=df['dms_dest'].replace(location_dict)

In [9]:
## Replacing foreign location id's with the country names

country_dict={801:'Canada',802:'Mexico',803:'Rest of America',804:'Europe',
              805:'Africa',806:'SW & Central Asia',807:'Eastern Asia',808:'SE Asia & Oceania'}
    
df['orig_country']=df['fr_orig'].replace(country_dict)
df['dest_country']=df['fr_dest'].replace(country_dict)

In [10]:
## Replacing trade type id's with their actual trade type names

mode_type_dict={1:'Truck',2:'Rail',3:'Water',4:'Air',
                5:'Multiple Modes and mail',6:'Pipeline',7:'Other and Unknown',8:'No domestic mode'}
df['dms_mode'].replace(mode_type_dict,inplace=True)
df['fr_inmode'].replace(mode_type_dict,inplace=True)
df['fr_outmode'].replace(mode_type_dict,inplace=True)

In [11]:
## Transforming region column - The first half of region column is stored as region. The second half is stored as state
## Eg : Birmingham-Hoover-Talladega, AL CFS Area ---> Birmingham-Hoover-Talladega -> Region, AL -> State

df['orig_region']=df['orig_region'].apply(lambda x:x.split(',')[0])
df['dest_region']=df['dest_region'].apply(lambda x:x.split(',')[0])

In [12]:
## Replacing commodity id's with their actual commodity names

commodity_dict={}
for i in range(len(commodity_labels_df)):
    id=commodity_labels_df.loc[i]['id']
    commodity=commodity_labels_df.loc[i]['commodity']
    commodity_dict.update({id:commodity})
    
df['commodity']=df['sctg2'].replace(commodity_dict)

In [13]:
df.head()

,fr_orig,dms_orig,dms_dest,fr_dest,fr_inmode,dms_mode,fr_outmode,sctg2,trade_type,dist_band,...,tmiles_2018,tmiles_2019,tmiles_2020,orig_region,dest_region,orig_state,dest_state,orig_country,dest_country,commodity
0,NaN,11,11,NaN,NaN,Truck,NaN,1,Domestic only,1,...,3.218410,3.351765,3.375547,Birmingham-Hoover-Talladega,Birmingham-Hoover-Talladega,AL,AL,NaN,NaN,Animals and Fish (live)
1,NaN,11,19,NaN,NaN,Truck,NaN,1,Domestic only,2,...,48.631191,50.646235,51.005589,Birmingham-Hoover-Talladega,Remainder of Alabama,AL,AL,NaN,NaN,Animals and Fish (live)
2,NaN,11,129,NaN,NaN,Truck,NaN,1,Domestic only,3,...,0.450066,0.468715,0.472040,Birmingham-Hoover-Talladega,Remainder of Florida,AL,FL,NaN,NaN,Animals and Fish (live)
3,NaN,11,131,NaN,NaN,Truck,NaN,1,Domestic only,2,...,2.177036,2.267241,2.283328,Birmingham-Hoover-Talladega,Atlanta-Athens-Clarke County-Sandy Springs,AL,GA,NaN,NaN,Animals and Fish (live)
4,NaN,11,139,NaN,NaN,Truck,NaN,1,Domestic only,2,...,1.274964,1.327792,1.337213,Birmingham-Hoover-Talladega,Remainder of Georgia,AL,GA,NaN,NaN,Animals and Fish (live)


## EDA

In [14]:
domestic_df=df[df['trade_type']=='Domestic only']
import_df=df[df['trade_type']=='Import']
export_df=df[df['trade_type']=='Export']

## Trade Type Composition

In [15]:
trade_type_count_df=pd.DataFrame(df['trade_type'].value_counts()).reset_index().rename(columns={'index':'Trade_Type','trade_type':'count'})

fig = go.Figure(data=[go.Pie(labels=trade_type_count_df['Trade_Type'], values=trade_type_count_df['count'], 
                             pull=[0,0.2,0.2],title=dict(text='Trade type composition',font=dict(size=20)))])
fig.update_layout(showlegend=False,margin=dict(l=0,r=0,b=0,t=20))
fig.update_traces(textinfo='percent+label')
fig.add_annotation(x=0.91,y=0.8,xref='paper',yref='paper',text='<b>Imports is slightly higher than exports</b>',showarrow=False)
fig.show()

In [16]:
export_countries_count=export_df.groupby('dest_country',as_index=False).agg({'dms_orig':'count'}).rename(columns={'dest_country':'destination country','dms_orig':'count'})
import_countries_count=import_df.groupby('orig_country',as_index=False).agg({'dms_orig':'count'}).rename(columns={'orig_country':'origin country','dms_orig':'count'})

fig=make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Pie(labels=import_countries_count['origin country'], values=import_countries_count['count'], 
                             pull=[0,0.2,0.2,0.2],title=dict(text='Countries from which USA imports',font=dict(size=20))),row=1,col=1)
fig.add_trace(go.Pie(labels=export_countries_count['destination country'], values=export_countries_count['count'], 
                             pull=[0,0.2,0.2,0.2],title=dict(text='Countries to which USA exports',font=dict(size=20))),row=1,col=2)
fig.update_traces(textinfo='label+percent')
fig.update_layout(margin=dict(l=0,r=0,b=0,t=30),showlegend=False)
fig.show()

## Value of goods traded across years in each trade type

In [17]:
trade_value_df=df.groupby('trade_type',as_index=False).agg({'current_value_2018':'sum','current_value_2019':'sum','current_value_2020':'sum'})
fig=px.bar(trade_value_df,x='trade_type',y=trade_value_df.columns[1:],barmode='group',title='Total value of goods traded in every trade type')
fig.show()

## Value of goods exported and imported for every country for 2018,2019 and 2020

In [18]:
import_value_df=import_df.groupby('orig_country',as_index=False).agg({'current_value_2018':'sum','current_value_2019':'sum','current_value_2020':'sum'})
export_value_df=export_df.groupby('dest_country',as_index=False).agg({'current_value_2018':'sum','current_value_2019':'sum','current_value_2020':'sum'})
fig=px.scatter(export_value_df,y='dest_country',x=export_value_df.columns[1:],title='Value of goods exported to all countries for 2018,2019  and 2020')
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')))
fig.update_layout(xaxis=dict(showgrid=False,title='Value (in million dollars)'))
fig.add_annotation(x=0.9,y=0.1,xref='paper',yref='paper',text='<b>For all places there is a decrease in value of exports every year</b>',
                  showarrow=False)
fig.show()

In [19]:
fig=px.scatter(import_value_df,y='orig_country',x=import_value_df.columns[1:],title='Value of goods imported from all countries for 2018,2019 and 2020')
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')))
fig.update_layout(xaxis=dict(showgrid=False,title='Value (in million dollars)'))
fig.add_annotation(x=0.6,y=0.9,xref='paper',yref='paper',text='<b>SouthEast Asia & Oceania region is the only place from which<br> value of goods imported increases every year</b>',
                  showarrow=False)
fig.add_annotation(x=0.9,y=0.1,xref='paper',yref='paper',text='<b>For all other places there is a decrease in value of imports every year</b>',
                  showarrow=False)
fig.show()

## Region based analysis

In [20]:
orig_export_df=export_df.groupby(['dest_country','orig_region'],as_index=False).agg({'current_value_2018':'mean','current_value_2019':'mean','current_value_2020':'mean'}).rename(columns={'current_value_2018':'avg_2018','current_value_2019':'avg_2019','current_value_2020':'avg_2020'})
orig_export_df=orig_export_df.sort_values(['avg_2018'],ascending=[False]).groupby('dest_country').head(1).sort_values(['dest_country','avg_2018'],ascending=[False,False])
fig=px.bar(orig_export_df,x='dest_country',y=orig_export_df.columns[3],barmode='group',text='orig_region',
          title='Top cities from which goods are exported to each country based on the value of goods exported in 2018')
fig.add_annotation(x=0,y=1,xref='paper',yref='paper',text='For example, <b>In the year 2018 on an average 30 million dollars worth of goods have been exported to Mexico from El Paso Las Cruces</b>',
                  showarrow=False)
fig.update_traces(textposition='inside')
fig.update_layout(margin=dict(l=0,r=0,b=0),yaxis=dict(title='Value in Million dollars'))
fig.show()

In [21]:
import_dest_df=import_df.groupby(['orig_country','dest_region'],as_index=False).agg({'current_value_2018':'mean','current_value_2019':'mean','current_value_2020':'mean'}).rename(columns={'current_value_2018':'avg_2018','current_value_2019':'avg_2019','current_value_2020':'avg_2020'})
import_dest_df=import_dest_df.sort_values(['avg_2018'],ascending=[False]).groupby('orig_country').head(1).sort_values(['orig_country','avg_2018'],ascending=[False,False])
fig=px.bar(import_dest_df,x='orig_country',y=import_dest_df.columns[3],barmode='group',text='dest_region',
          title='Top cities to which goods are imported from each country based on the value of goods imported in 2018')
fig.add_annotation(x=0,y=1,xref='paper',yref='paper',text='For example, <b>In the year 2018 on an average 48 million dollars worth of goods have been imported from Mexico to Detroit-Warren-Ann harbour</b>',
                  showarrow=False)
fig.update_traces(textposition='inside')
fig.update_layout(margin=dict(l=0,r=0,b=0),yaxis=dict(title='Value in Million dollars'))
fig.show()

## Top Origin-Destination pairs for domestic trade

In [22]:
domestic_value_avg_df=domestic_df.groupby(['orig_region','dest_region'],as_index=False).agg({'current_value_2018':'mean','current_value_2019':'mean','current_value_2020':'mean'})
for i in range(len(domestic_value_avg_df)):
    if domestic_value_avg_df['orig_region'][i]==domestic_value_avg_df['dest_region'][i]:
        domestic_value_avg_df.drop(i,axis=0,inplace=True)
domestic_value_avg_df=domestic_value_avg_df.sort_values('current_value_2018',ascending=False)[:20]

domestic_value_sum_df=domestic_df.groupby(['orig_region','dest_region'],as_index=False).agg({'current_value_2018':'sum','current_value_2019':'sum','current_value_2020':'sum'})
for i in range(len(domestic_value_sum_df)):
    if domestic_value_sum_df['orig_region'][i]==domestic_value_sum_df['dest_region'][i]:
        domestic_value_sum_df.drop(i,axis=0,inplace=True)
domestic_value_sum_df=domestic_value_sum_df.sort_values('current_value_2018',ascending=False)[:20]

fig=make_subplots(rows=2,cols=1,specs=[[{'type':'table'}],
                                      [{'type':'table'}]],
                 subplot_titles=['Top Origin-Destination region pairs for domestic trade(Based on average value of goods exported)',
                                'Top Origin-Destination region pairs for domestic trade(Based on total value of goods exported)'])

fig.add_trace(go.Table(header=dict(values=['Origin Region','Destination Region','Value 2018(million $)','Value 2019(million $)','Value 2020(million $)'],font=dict(size=10),align='left'),
                       cells=dict(values=[domestic_value_avg_df[k].tolist() for k in domestic_value_avg_df.columns],align = 'left')),row=1,col=1)
fig.add_trace(go.Table(header=dict(values=['Origin Region','Destination Region','Value 2018(million $)','Value 2019(million $)','Value 2020(million $)'],font=dict(size=10),align='left'),
                       cells=dict(values=[domestic_value_sum_df[k].tolist() for k in domestic_value_sum_df.columns],align = 'left')),row=2,col=1)

fig.update_layout(height=800,width=1000)
fig.add_annotation(x=0,y=0.5,xref='paper',yref='paper'
                   ,text='<b>These tables doesnt take into consideration trades with same origin and destination regions i.e Newyork-Newyork kind of trades</b>',
                  showarrow=False)
fig.show()

From the above tables we could find that for all trading routes there is a decrease in average value of goods traded from 2018 to 2020 and the obvious reason is pandemic. But for one particular route <b>Sacramento-Roseville ---> San Jose-San Francisco-Oakland</b> there is an increase in value of goods traded. We can inspect it further

For this particular trade route the increase in value of goods traded has been predominantly driven by the trading of commodities such as <b>Alcohol & Tobacco, Pharmaceutical Products, Agricultural products</b> etc.

But Alcoholic beverages, Pharmaceutical products have seen increase trading volumes across all routes. One product that has seen increased trading in this particular route is <b>Agricultural products</b>.

In [23]:
a=domestic_df[(domestic_df['orig_region']=='Sacramento-Roseville')&(domestic_df['dest_region']=='San Jose-San Francisco-Oakland')]
a=a.groupby(['orig_region','dest_region','commodity'],as_index=False).agg({'current_value_2018':'mean','current_value_2019':'mean','current_value_2020':'mean'})
a['pct_2018_2020']=100*(a['current_value_2020']-a['current_value_2018'])/a['current_value_2018']
a.sort_values('pct_2018_2020',ascending=False)[:10].reset_index().drop('index',axis=1)

,orig_region,dest_region,commodity,current_value_2018,current_value_2019,current_value_2020,pct_2018_2020
0,Sacramento-Roseville,San Jose-San Francisco-Oakland,Alcoholic Beverages and Denatured Alcohol,84.104370,85.209193,98.412172,17.011962
1,Sacramento-Roseville,San Jose-San Francisco-Oakland,Pharmaceutical Products,8617.889546,9109.081992,9525.063853,10.526641
2,Sacramento-Roseville,San Jose-San Francisco-Oakland,Tobacco Products,0.708874,0.723555,0.777412,9.668573
3,Sacramento-Roseville,San Jose-San Francisco-Oakland,"Agricultural Products (excludes Animal Feed, C...",224.982184,214.698097,245.816676,9.260508
4,Sacramento-Roseville,San Jose-San Francisco-Oakland,Natural Sands,2.161713,2.185548,2.270137,5.015652
5,Sacramento-Roseville,San Jose-San Francisco-Oakland,Gravel and Crushed Stone (excludes Dolomite an...,0.083120,0.083969,0.087157,4.856833
6,Sacramento-Roseville,San Jose-San Francisco-Oakland,"Meat, Poultry, Fish, Seafood, and Their Prepar...",318.906611,332.042795,332.426199,4.239356
7,Sacramento-Roseville,San Jose-San Francisco-Oakland,Fertilizers,6.422140,7.234081,6.692156,4.204455
8,Sacramento-Roseville,San Jose-San Francisco-Oakland,Wood Products,307.193881,305.191217,319.464124,3.994299
9,Sacramento-Roseville,San Jose-San Francisco-Oakland,"Other Prepared Foodstuffs, Fats and Oils",452.737667,464.464966,463.641533,2.408429


## Highest and Lowest commodities traded based on frequency of trade

In [24]:
top_commodity=pd.DataFrame(df['commodity'].value_counts()).reset_index().rename(columns={'index':'commodity','commodity':'count'})[:10]
bottom_commodity=pd.DataFrame(df['commodity'].value_counts()).reset_index().rename(columns={'index':'commodity','commodity':'count'})[-10:]
commodity_count_df=pd.concat([top_commodity,bottom_commodity],axis=0).reset_index().drop('index',axis=1)

fig=px.bar(commodity_count_df,y='commodity',x='count',orientation='h',text='commodity')
fig.add_hline(y=9.5, line_width=3, line_dash="dash", line_color="green")
fig.add_annotation(x=1,y=0.7,xref='paper',yref='paper',text='<b>Lowest 10 commodities traded</b>',showarrow=False)
fig.add_annotation(x=1,y=0.3,xref='paper',yref='paper',text='<b>Top 10 commodities traded</b>',showarrow=False)
fig.update_layout(yaxis=dict(showticklabels=False))

## Highest and Lowest commodities traded based on value of goods(2020)

In [25]:
top_commodity_value_df=df.groupby('commodity',as_index=False).agg({'current_value_2020':'sum'}).sort_values('current_value_2020',ascending=False)[:10]
bottom_commodity_value_df=df.groupby('commodity',as_index=False).agg({'current_value_2020':'sum'}).sort_values('current_value_2020',ascending=False)[-10:]
commodity_value_df=pd.concat([top_commodity_value_df,bottom_commodity_value_df],axis=0).reset_index().drop('index',axis=1)

fig=px.bar(commodity_value_df,y='commodity',x='current_value_2020',orientation='h',text='commodity')
fig.add_hline(y=9.5, line_width=3, line_dash="dash", line_color="green")
fig.add_annotation(x=1,y=0.7,xref='paper',yref='paper',text='<b>Lowest 10 commodities traded based on value</b>',showarrow=False)
fig.add_annotation(x=1,y=0.3,xref='paper',yref='paper',text='<b>Top 10 commodities traded based on value</b>',showarrow=False)
fig.update_layout(yaxis=dict(showticklabels=False))

## Top commodities imported and exported for each country based on the count as well as value of goods

In [26]:
import_commodity_df=import_df.groupby(['orig_country','commodity'],as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
import_commodity_df=import_commodity_df.sort_values(['orig_country','count'],ascending=[True,False]).groupby('orig_country').head(1)

import_commodity_value_df=import_df.groupby(['orig_country','commodity'],as_index=False).agg({'current_value_2020':'mean'})
import_commodity_value_df=import_commodity_value_df.sort_values(['orig_country','current_value_2020'],ascending=[True,False]).groupby('orig_country').head(3)


fig=make_subplots(rows=2,cols=1,specs=[[{'type':'table'}],
                                      [{'type':'table'}]],
                 subplot_titles=['Top Commodities imported from every country(Based on count)',
                                'Top 3 Commodities imported from every country(Based on value of goods for the year 2020)'])

fig.add_trace(go.Table(header=dict(values=['Origin Country','Destination Region','count'],font=dict(size=10),align='left'),
                       cells=dict(values=[import_commodity_df[k].tolist() for k in import_commodity_df.columns],align = 'left')),row=1,col=1)
fig.add_trace(go.Table(header=dict(values=['Origin Region','Destination Region','Current value 2020 (in million dollars)'],font=dict(size=10),align='left'),
                       cells=dict(values=[import_commodity_value_df[k].tolist() for k in import_commodity_value_df.columns],align = 'left')),row=2,col=1)

fig.update_layout(height=600,width=1000)

fig.show()

In [27]:
export_commodity_df=export_df.groupby(['dest_country','commodity'],as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
export_commodity_df=export_commodity_df.sort_values(['dest_country','count'],ascending=[True,False]).groupby('dest_country').head(1)

export_commodity_value_df=export_df.groupby(['dest_country','commodity'],as_index=False).agg({'current_value_2020':'mean'})
export_commodity_value_df=export_commodity_value_df.sort_values(['dest_country','current_value_2020'],ascending=[True,False]).groupby('dest_country').head(3)

fig=make_subplots(rows=2,cols=1,specs=[[{'type':'table'}],
                                      [{'type':'table'}]],
                 subplot_titles=['Top Commodities exported to every country(Based on count)',
                                'Top 3 Commodities exported to every country(Based on value of goods for the year 2020)'])

fig.add_trace(go.Table(header=dict(values=['Origin Country','Destination Region','count'],font=dict(size=10),align='left'),
                       cells=dict(values=[export_commodity_df[k].tolist() for k in export_commodity_df.columns],align = 'left')),row=1,col=1)
fig.add_trace(go.Table(header=dict(values=['Origin Region','Destination Region','Current value 2020 (in million dollars)'],font=dict(size=10),align='left'),
                       cells=dict(values=[export_commodity_value_df[k].tolist() for k in export_commodity_value_df.columns],align = 'left')),row=2,col=1)

fig.update_layout(height=600,width=1000)

fig.show()

## Top commodities traded domestically based on value of goods

If we consider top commodites for import and export based on value of goods, <b>crude petroleum</b> occupies the top most position

In [28]:
commodity_value_domestic_df=domestic_df.groupby(['commodity'],as_index=False).agg({'current_value_2018':'mean','current_value_2019':'mean','current_value_2020':'mean'}).sort_values('current_value_2018',ascending=False)[:15]
fig=px.bar(commodity_value_domestic_df,x='commodity',y=commodity_value_domestic_df.columns[1:],barmode='group')
fig.add_annotation(x=0.1,y=0.8,xref='paper',yref='paper',text='<b>Pharmaceuticals, Tobacco, Alcohol and Meat products</b> are the categories <br>where there is a slight increase in value of goods over years',
                  showarrow=False)
fig.update_layout(margin=dict(l=0,r=0),yaxis=dict(title='Value in million dollars'))
fig.show()

## Top 5 commodites traded in every trade type

In [29]:
trade_commodity_groupby=df.groupby(['trade_type','commodity'],as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
trade_commodity_groupby=trade_commodity_groupby.sort_values(['trade_type','count'],ascending=[True,False]).groupby('trade_type').head(5)
fig=px.treemap(trade_commodity_groupby,path=['trade_type','commodity'],values='count',title='Top commodities traded in every trade type')
fig.update_layout(margin=dict(l=0,b=0,r=0,t=40))
fig.add_annotation(x=0.1,y=1,xref='paper',yref='paper',text='<b>Machinery, Electronic items, precision instruments are the most traded in every category</b>',showarrow=False)
fig.show()

## Commodities and trade type

In [30]:
commodity_trade_groupby=df.groupby(['commodity','trade_type'],as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
commodity_trade_groupby=commodity_trade_groupby.sort_values(['commodity','count'],ascending=[True,False]).groupby('commodity').head()
commodity_trade_groupby['pct']=commodity_trade_groupby.groupby('commodity')['count'].apply(lambda x:round(x*100/x.sum(),2))

fig=px.bar(commodity_trade_groupby,y='commodity',color='trade_type',x='pct',orientation='h',
           width=1000,height=1000,text='pct',title='Commodities categorized by trade type')
fig.update_layout(margin=dict(l=0,r=0,b=0))
fig.add_annotation(x=-2.1,y=0.82,xref='paper',yref='paper',
                   text='<b>Crude Petroleum is majorly imported while <br>coal is majorly exported<br><br>Cereal grains, fuel oils, wooden logs <br>and waste scrap are predominantly exported</b>',
                  showarrow=False)
fig.show()

## Preferred Transportation mode in every trade type

In [31]:
dms_mode_groupby=df.groupby('dms_mode',as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
fr_inmode_groupby=df.groupby('fr_inmode',as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
fr_outmode_groupby=df.groupby('fr_outmode',as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})

fig=make_subplots(rows=3,cols=1,subplot_titles=['Transportation modes used during domestic trade movement',
                                               'Transportation modes used during import',
                                               'Transportation modes used during export'])
fig.add_trace(go.Bar(x=dms_mode_groupby['dms_mode'],y=dms_mode_groupby['count'],name='Domestic'),row=1,col=1)
fig.add_trace(go.Bar(x=fr_inmode_groupby['fr_inmode'],y=fr_inmode_groupby['count'],name='Import'),row=2,col=1)
fig.add_trace(go.Bar(x=fr_outmode_groupby['fr_outmode'],y=fr_outmode_groupby['count'],name='Export'),row=3,col=1)
fig.add_annotation(x=0.3,y=0.95,xref='paper',yref='paper',
                   text='<b>Truck is the most used transportation method in domestic travel.</b>',
                  showarrow=False)
fig.add_annotation(x=0.25,y=0.5,xref='paper',yref='paper',
                   text='<b>Air and water is the most used transportation method in export and import.</b>',
                  showarrow=False)
fig.update_layout(height=1000,width=1000,margin=dict(l=0,r=0,b=0,t=30))

## Top commodities traded via different transportation modes in different trades

<b>Truck</b> is a major transportation mode in domestic trade movement.

Irrespective of trade type, <b>electical and electornic equipments</b> cover a major portion in air, water and truck transportation modes.
 
<b>Machinery</b> is majorly imported or exported via water ways compared with airways.

In [32]:
a=df.groupby(['dms_mode','commodity'],as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
a=a.sort_values(['dms_mode','count'],ascending=[True,False]).groupby('dms_mode').head(5)
fig=px.bar(a,x='dms_mode',y='count',color='commodity',title='Top commodities traded via transportation modes in all doemstic trade')
fig.show()

In [33]:
a=df.groupby(['fr_inmode','commodity'],as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
a=a.sort_values(['fr_inmode','count'],ascending=[True,False]).groupby('fr_inmode').head(5)
fig=px.bar(a,x='fr_inmode',y='count',color='commodity',title='Top commodities traded via transportation modes in imports')
fig.show()

In [34]:
a=df.groupby(['fr_outmode','commodity'],as_index=False).agg({'dms_orig':'count'}).rename(columns={'dms_orig':'count'})
a=a.sort_values(['fr_outmode','count'],ascending=[True,False]).groupby('fr_outmode').head(5)
fig=px.bar(a,x='fr_outmode',y='count',color='commodity',title='Top commodities traded via transportation modes in exports')
fig.show()

## Value of goods transported via multiple modes across every trade type

<b>Trucks</b> are predominantly used in <b>domestic trade</b> and obviously the value of goods they ship is also higher.

Goods transported via <b>waterways possess more value in import based trade and export based trade</b> while airways closely follows waterways in exports. The below plot takes into account value of goods in 2020.

In [35]:
dms_mode_value=df.groupby('dms_mode',as_index=False).agg({'current_value_2020':'sum'})
fr_inmode_value=df.groupby('fr_inmode',as_index=False).agg({'current_value_2020':'sum'})
fr_outmode_value=df.groupby('fr_outmode',as_index=False).agg({'current_value_2020':'sum'})

fig=make_subplots(rows=3,cols=1,subplot_titles=['Value of goods transported via different modes in domestic trade',
                                               'Value of goods transported via different modes in imports',
                                               'Value of goods transported via different modes in exports'])
fig.add_trace(go.Bar(x=dms_mode_value['dms_mode'],y=dms_mode_value['current_value_2020'],name='Domestic'),row=1,col=1)
fig.add_trace(go.Bar(x=fr_inmode_value['fr_inmode'],y=fr_inmode_value['current_value_2020'],name='Import'),row=2,col=1)
fig.add_trace(go.Bar(x=fr_outmode_value['fr_outmode'],y=fr_outmode_value['current_value_2020'],name='Export'),row=3,col=1)

fig.update_layout(height=1000,width=1000,margin=dict(l=0,r=0,b=0,t=30))

## SUMMARY

<b>New York</b> acts as an important region via which goods are traded.

<b>New York-Newark to  Philadelphia-Reading-Camden</b> is an important route via which a lot of trade movement occurs.

<b>Los Angeles-San Deigo</b> trade route is the top domestic trade route based on the average value of goods shipped between them while Newyork-Philadelphia is the top domestic trade route based on total value of goods shipped.

In this given dataset, <b>38% accounts for imports while 36% accounts for exports and 25% covers domestic trade.</b>

<b>Europe, Eastern Asia and Canada</b> are top regions from which USA exports and imports.

In the given dataset <b>based on the value of goods traded, domestic trade occupies the first spot</b> while imports and exports lags behind.

For all years, for all regions there is a decrease in value of goods exported and imported while <b>SouthEast Asia and Oceania is the only region where there is a increase in value of goods imported from 2018 to 2020.</b>

<b>Based on value of goods, Electrical and electronic equipments is the top commodity to be traded while Monumental or Building stones is the lowest commodity traded</b>.

<b>Based on weight of goods shipped, Coal and Petroleum products is the top commodity while tobacco products is the lowest commodity</b>.

<b>Textiles or Electrical equipments is the most imported commodity across all regions</b> while <b>Machinery</b> is the most imported commodity from <b>Europe</b>.

<b>Crude Petroleum is the costliest commodity</b> either imported or exported from/to almost all regions.

<b>Crude Petroleum is majorly imported<b> while <b>coal is majorly exported</b>.
    
<b>Cereal grains, fuel oils, wooden logs and waste scrap are predominantly exported</b>.

<b>Trucks are the preferred mode of transportation in domestic trade while Airways and Waterways are the preferred mode in import and exports</b>.

During <b>imports and exports waterways carries the heaviest goods</b>.